<font face="Calibri" size="2"> <i>SBAE - Notebook Series - Part 2, version 0.3, October 2022. Andreas Vollrath, UN-Food and Agricultural Organization, Rome</i>
</font>

![title](images/header.png)

# II - SBAE Time-Series Extraction & Change Detection
### Extract various time-series outputs for point data from Google Earth Engine
-------

This notebook takes you through the process of extracting outputs from various time-series change detection algorithms and structure them in a so-called data-frame (e.g. tabular structure).

### 1 - Import libs

**ONLY EXECUTE THIS CELL**

In [1]:
import time 
from pathlib import Path
from datetime import datetime as dt

import ee
from geemap import Map
# initialize EE    
try:
    ee.Initialize(opt_url='https://earthengine-highvolume.googleapis.com')
except:
    ee.Authenticate()
    ee.Initialize(opt_url='https://earthengine-highvolume.googleapis.com')

import helpers as h

/usr/local/lib/python3.8/dist-packages/pandas/core/computation/expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
/usr/local/lib/python3.8/dist-packages/bfast/__init__.py:10: UserWarning: cupy is not available in this environment, GPU fonctionnalities won't be available
  warn("cupy is not available in this environment, GPU fonctionnalities won't be available")


### 2 - Basic Input Variables

**FILL IN YOUR INPUTS**

In [2]:
# Point Feature Collection from where to extract points
#fc = ee.FeatureCollection("users/fao-dannunzio/civ_erp_5km_pts_exo")

# the column/property of a unique point identifier in your dataset
point_id_name = 'point_id'

#### 2c - Time-series parameter settings

In [3]:
# start of calibration period (mainly for bfast)
start_calibration = "1985-01-01"  # YYYY-MM-DD format

# Actual period of interest, i.e. monitoring period
start_monitor =     "2000-01-01"  # YYYY-MM-DD format
end_monitor =       "2017-01-01"  # YYYY-MM-DD format

# Directory where output and temp files will go
outdir = 'erp_5km'  # goes to module_results/sbae_point_analysis if set to None

# Select algorithms to run (Treu or False)
cusum_deforest =  True
bfast_monitor =   True
bs_slope =        True
ts_metrics =      True
ccdc =            True
landtrendr =      True
jrc_nrt =         True
global_products = True

# select the bands to extract
bands = ['green', 'red', 'nir', 'swir1', 'swir2', 'ndfi'] # other choices: ndfi, ndmi, mndwi, brightness, greenness, wetness

# select the band for univariate ts-analysis (has to be inside bands list)
ts_band = 'ndfi'

# select the resolution to which the satellite data will be resized.
scale = 70  # in meters (70 m is half ha, relates to FAO forest definition)

### DO NOT CHANGE YET ###
satellite='Landsat'  # this is going to be Surface Reflactance, Collection 2, Tier 1 data only
max_cloud_cover = 75  # in percentage (0-100)

### 3- Algorithm parameter settings

**Edit for advanced users, otherwise just execute**

In [4]:
# landsat related parameters
lsat_params = {
    'l9':     True,
    'l8':     True,
    'l7':     True,
    'l5':     True,
    'l4':     True,
    'brdf':   True,
    'bands':  bands,
    'max_cc': max_cloud_cover
}

# bfast parameters
bfast_params = {
    'run':           bfast_monitor,
    'start_monitor': start_monitor, 
    'freq':          365,
    'k':             3, 
    'hfrac':         0.25, 
    'trend':         True, 
    'level':         0.05, 
    'backend':       'python'
}

# cusum parameters
cusum_params = {
    'run':              cusum_deforest,
    'nr_of_bootstraps': 1000
}

# slope parameters
bs_slope_params = {
    'run':              bs_slope,
    'nr_of_bootstraps': 1000
}

# time-series metrics
ts_metrics_params = {
    'run':              ts_metrics,
    'outlier_removal':  False,
    'z_threshhold':     3
}

# ccdc parameters
ccdc_params = {
    'run':                   ccdc,
    'breakpointBands':       ['green', 'red', 'nir', 'swir1', 'swir2'],
    'tmaskBands':            ['green', 'swir2'],
    'minObservations':       6,
    'chiSquareProbability':  .99,
    'minNumOfYearsScaler':   1,
    'dateFormat':            2,
    'lambda':                20,
    'maxIterations':         1000
}


landtrendr_params = { 
        'run':                    landtrendr,
        'maxSegments':            6,
        'spikeThreshold':         0.9,
        'vertexCountOvershoot':   3,
        'preventOneYearRecovery': True,
        'recoveryThreshold':      0.25,
        'pvalThreshold':          0.05,
        'bestModelProportion':    0.75,
        'minObservationsNeeded':  3
}

jrc_nrt_params = {
    'run': jrc_nrt
}

# global products parameters
global_products = {
    'run':                      global_products,
    'gfc':                      True,     # will include tree-cover 2000, loss, gain, lossyear
    'tmf':                      True,    # will include deforestation and degradation year for tropical moist forests
    'tmf_years':                True,    # will include classes per year - according to the monitor period
    'esa_lc20':                 True,    # will include ESA LandCover Product class
    'copernicus_lc':            True,    # will include ESA LandCover Product class - acording to the monitoring years
    'esri_lc':                  True,    # will include the classes from ESRI World Cover 2020
    'lang_tree_height':         True,    # returns the Tree Height from Lang et al 2022
    'potapov_tree_height':      True,    # returns the tree height from Potapov et al. 2019 
    'elevation':                True,    # returns elevation, slope and aspect
    'dynamic_world_tree_prob':  True,    # returns Min, Max, Mean and StdDev of the trees probability for the monitoring period
    'dynamic_world_class_mode': True     # returns the mode of the class for the monitoring period   
}

### DO NOT CHANGE ###
### GATHER ALL INFO INTO A DICT #####
config_dict = {
    'work_dir':                         outdir,
    'workers':                          10,
    'max_points_per_chunk':             250,
    'grid_size_levels':                 [4, 2, 1, 0.5, 0.25, 0.125, 0.075],  # definition of chunk sizes in degrees  
    'lsat_params':                      lsat_params,
    'ts_params': {
        'start_calibration':            start_calibration,
        'start_monitor':                start_monitor,
        'end_monitor':                  end_monitor,
        'point_id':                     point_id_name,
        'bands':                        bands,
        'ts_band':                      ts_band,
        'satellite':                    satellite,
        'scale':                        scale,
        'max_cc':                       max_cloud_cover,
        'outlier_removal':              True,
        'smooth_ts':                    True       
    },    
    'bfast_params':                     bfast_params,
    'cusum_params':                     cusum_params,
    'bs_slope_params':                  bs_slope_params,
    'ts_metrics_params':                ts_metrics_params,
    'ccdc_params':                      ccdc_params,
    'landtrendr_params':                landtrendr_params,
    'jrc_nrt_params':                   jrc_nrt_params,
    'global_products':                  global_products
}

### 4 - Run the time-series data extraction

**Execute only**

In [5]:
h.get_change_data(fc, config_dict)

 Setting up the processing pipeline. This may take a moment
 Trying to delete temporary Earth Engine folder/assets from previous runs.
 Creating temporary folder
 Nr of plots to process: 1860
 --------------------------------------------------------------------------------------------
 Splitting the aoi in chunks for parallel processing (Level 1).
 Parallelizing on chunks of 4x4 degrees, totalling in 1 chunks.
 1860 points left to process.
 --------------------------------------------------------------------------------------------
 More than 250 points in chunk 1. Considering respective points at smaller chunk size level.
 --------------------------------------------------------------------------------------------
 Splitting the aoi in chunks for parallel processing (Level 2).
 Parallelizing on chunks of 2x2 degrees, totalling in 4 chunks.
 1860 points left to process.
 --------------------------------------------------------------------------------------------
 More than 250 points i

/usr/local/lib/python3.8/dist-packages/xarray/core/variable.py:1899: RuntimeWarning: All-NaN slice encountered
  data = func(self.data, axis=axis, **kwargs)
/home/sepal-user/.local/lib/python3.8/site-packages/nrt/monitor/__init__.py:522: UserWarning: 4 time-series were shorter than 1.5x the number of regressors and were masked.
  warnings.warn(f'{amount} time-series were shorter than 1.5x the '
/home/sepal-user/.local/lib/python3.8/site-packages/nrt/monitor/__init__.py:522: UserWarning: 4 time-series were shorter than 1.5x the number of regressors and were masked.
  warnings.warn(f'{amount} time-series were shorter than 1.5x the '
/home/sepal-user/.local/lib/python3.8/site-packages/nrt/monitor/__init__.py:522: UserWarning: 4 time-series were shorter than 1.5x the number of regressors and were masked.
  warnings.warn(f'{amount} time-series were shorter than 1.5x the '
2022-12-02 21:41:57.558936: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_D

 Chunk 14 with 4 points done in: 0:04:52.230640


/usr/local/lib/python3.8/dist-packages/bfast/monitor/python/base.py:255: RuntimeWarning: invalid value encountered in sqrt
  sigma = np.sqrt(np.sum(y_error[:ns] ** 2) / (ns - (2 + 2 * self.k)))
/usr/local/lib/python3.8/dist-packages/bfast/monitor/python/base.py:255: RuntimeWarning: divide by zero encountered in double_scalars
  sigma = np.sqrt(np.sum(y_error[:ns] ** 2) / (ns - (2 + 2 * self.k)))
/usr/local/lib/python3.8/dist-packages/xarray/core/variable.py:1899: RuntimeWarning: All-NaN slice encountered
  data = func(self.data, axis=axis, **kwargs)
/home/sepal-user/.local/lib/python3.8/site-packages/nrt/monitor/__init__.py:522: UserWarning: 5803 time-series were shorter than 1.5x the number of regressors and were masked.
  warnings.warn(f'{amount} time-series were shorter than 1.5x the '
/home/sepal-user/.local/lib/python3.8/site-packages/nrt/monitor/__init__.py:522: UserWarning: 5803 time-series were shorter than 1.5x the number of regressors and were masked.
  warnings.warn(f'{amoun

 Chunk 13 with 12 points done in: 0:09:01.485128


...

 Chunk 10 with 101 points done in: 0:10:30.331922


/usr/local/lib/python3.8/dist-packages/bfast/monitor/python/base.py:255: RuntimeWarning: invalid value encountered in sqrt
  sigma = np.sqrt(np.sum(y_error[:ns] ** 2) / (ns - (2 + 2 * self.k)))
/usr/local/lib/python3.8/dist-packages/bfast/monitor/python/base.py:255: RuntimeWarning: divide by zero encountered in double_scalars
  sigma = np.sqrt(np.sum(y_error[:ns] ** 2) / (ns - (2 + 2 * self.k)))
/usr/local/lib/python3.8/dist-packages/xarray/core/variable.py:1899: RuntimeWarning: All-NaN slice encountered
  data = func(self.data, axis=axis, **kwargs)
/home/sepal-user/.local/lib/python3.8/site-packages/nrt/monitor/__init__.py:522: UserWarning: 2425 time-series were shorter than 1.5x the number of regressors and were masked.
  warnings.warn(f'{amount} time-series were shorter than 1.5x the '
/home/sepal-user/.local/lib/python3.8/site-packages/nrt/monitor/__init__.py:522: UserWarning: 2425 time-series were shorter than 1.5x the number of regressors and were masked.
  warnings.warn(f'{amoun

 Chunk 8 with 121 points done in: 0:18:01.520961
 Chunk 5 with 88 points done in: 0:18:34.360993
 Chunk 6 with 126 points done in: 0:19:37.268977
 Chunk 9 with 127 points done in: 0:22:11.570813
 Chunk 4 with 100 points done in: 0:25:37.929275
 Chunk 3 with 111 points done in: 0:27:10.799416


/usr/local/lib/python3.8/dist-packages/geopandas/io/file.py:362: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  pd.Int64Index,


 Deleting temporary files
 Deleting temporary EE assets...
 Processing has been finished successfully. Check for final_results files in your output directory.
